In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras

2023-10-08 18:06:40.720892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
"""
En esa celda se importan los datos desde el archivo genre_dataset.txt. Las columnas track_id, artist_name y title se ignoran (no se importan) y se quitan todas las filas cuyo género 
no sea dance and electronica', 'jazz and blues', 'soul and reggae', 'punk' o 'metal'. Los datos con solo los géneros deseados se guardan en DatosTarea. Finalmente se asigna un número 
a cada género y se cambia el valor del género por su respectivo número en la columna de 'genre. Por ejemplo, todos los 'dance and electronica' se cambian por un 0 y todos los 'punk' 
se cambian por un 3.
"""
DatosFull = pd.read_csv('genre_dataset.txt', skiprows = 9, usecols = lambda x: x not in ('track_id', 'artist_name', 'title'))
if '%genre' in DatosFull.keys():
    columna_genero = DatosFull.pop('%genre')
    DatosFull.insert(0, 'genre', columna_genero)
#fin if 
#print(set(DatosFull['genre']))
#{'punk', 'hip-hop', 'soul and reggae', 'metal', 'folk', 'pop', 'classic pop and rock', 'classical', 'jazz and blues', 'dance and electronica'}
generos_a_estudiar = {'dance and electronica': 0, 'jazz and blues': 1, 'soul and reggae': 2, 'punk': 3, 'metal': 4}
DatosTarea = DatosFull.loc[DatosFull['genre'].isin(tuple(generos_a_estudiar.keys()))]
indices_genero = np.array(map(lambda x: generos_a_estudiar[x], DatosTarea['genre']))
del DatosTarea['genre']
DatosTarea.insert(0, 'genre', indices_genero)

In [3]:
"""
En esta celda dividimos los datos en entrenamiento, validación y test
"""
#NO olvide borrar el random state
features = list(DatosTarea.keys())
features.remove('genre')
X_train, X_temp, y_train, y_temp = train_test_split(DatosTarea[features], DatosTarea['genre'], test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

In [4]:

"""
Aquí secuestramos la función para normalizar que habíamos hecho en la tarea 1. Y luego la usamos para normalizar los datos de la tarea 2.
Los datos de validación y test se normalizaron con la media y la desviación de los datos de entrenamiento
"""
parametros_train = dict(map(lambda x: (x, {'media': np.mean(X_train[x]), 'desviacion': np.std(X_train[x])}), X_train.keys()))
def normalizar(dataSet, features, parametros = []):
    """
    Esta función resta la media y luego divide entre la desviación estandar de cada dato en cada columna. Si se le especifica la media y la desviación en los parámetros 
    Usa esa media y desviación dada, de lo contrario, calcula la media y la desviación de cada columna especificada en "features"
    @input: dataSet <pd.DataFrame>: Tabla de datos a la que se le va a dar normalización a la columans dadas
    @input: features <list>: lista de "features" o columnas que se van a normalizar
    @input: (opcional) parametros <dict>: En caso de que se quiera normalizar con una media y una desviación dada se especifican en un diccionario así: {'mileage':{'media': 2.9, 'desviacion': 7}}
    """
    if len(parametros) == 0:
        for feature in features:
            media = np.mean(dataSet[feature])
            desviacion = np.std(dataSet[feature])
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
    #fin if 
    else:
        for feature in features:
            media = parametros[feature]['media']
            desviacion = parametros[feature]['desviacion']
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
#fin función
normalizar(X_train, features)
normalizar(X_val, features, parametros_train)
normalizar(X_test, features, parametros_train)

In [5]:
def devolver_vector(y_i, tamaño):
    """
    Esta función recibe un número entero positivo y_i y otro entero positivo indicando el tamaño del vector a devolver. Este devuelve un vector de ceros excepto en la posición y_i, 
    donde devuelve 1. Por ejemplo, si se llama la función así: devolver_vector(2,5) esta devolverá: [0, 0, 1, 0, 0]
    @input y_i <int>: Posición a la que se le desea colocar 1 en el vector a devolver
    @input tamaño <int>: Tamaño del vector a devolver
    @output <np.array>: Vector de todos ceros, excepto con un 1 en la posición especificada en y_i
    """
    vec = np.zeros(tamaño)
    vec[y_i] = 1
    return vec
#fin funcion

def prcesar_datos_en_y(y):
    """
    Esta función recibe un vector de respuesta y (los cuales son el género de cada canción) y devuelve una matriz de n_generos*n_datos en donde cada fila es un dato. Por ejemplo, 
    si se usa con tres datos, de los cuales uno es jazz, otro es punk y otro es metal, es decir, y = [1, 3, 4], el valor de procesar_datos_en_y([1,3,4]) dará como resultado una matriz así: 
    [[0,1,0,0,0], [0,0,0,1,0],[0,0,0,0,1]]
    @input y <lista o np.array o pd.DataFrame>: Vector que indica el género musical en los datos
    @output y_proc <np.array>: Matriz de ceros y unos que contiene la misma información
    """
    tamaño_vec = max(y) + 1
    y_proc = list(map(lambda x: devolver_vector(x, tamaño_vec),y))
    return np.array(y_proc)
#fin funcion

In [6]:
y_pro_train = prcesar_datos_en_y(y_train)
y_pro_val = prcesar_datos_en_y(y_val)
y_pro_test = prcesar_datos_en_y(y_test)
N_generos = max(y_train)
print(y_pro_train)

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [7]:
el_modelo = keras.models.Sequential()
el_modelo.add(keras.layers.Dense(int(len(features)/2), activation = 'relu', input_shape=(len(features))))
el_modelo.add(keras.layers.Dense(N_generos, activation = 'softmax'))
el_modelo.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy')
el_modelo.summary()

: 